In [1]:
import os
import argparse
from mindspore import context
from mindspore.train.callback import LossMonitor, SummaryCollector 

parser = argparse.ArgumentParser(description='MindSpore LeNet Example')
parser.add_argument('--device_target', type=str, default="CPU", choices=['Ascend', 'GPU', 'CPU'])

args = parser.parse_known_args()[0]
context.set_context(mode=context.GRAPH_MODE, device_target=args.device_target)


In [2]:
import os
import requests

def download_dataset(dataset_url, path):
    filename = dataset_url.split("/")[-1]
    save_path = os.path.join(path, filename)
    if os.path.exists(save_path):
        return
    if not os.path.exists(path):
        os.makedirs(path)
    res = requests.get(dataset_url, stream=True, verify=False)
    with open(save_path, "wb") as f:
        for chunk in res.iter_content(chunk_size=512):
            if chunk:
                f.write(chunk)

train_path = "datasets/MNIST_Data/train"
test_path = "datasets/MNIST_Data/test"

download_dataset("https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/train-labels-idx1-ubyte", train_path)
download_dataset("https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/train-images-idx3-ubyte", train_path)
download_dataset("https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/t10k-labels-idx1-ubyte", test_path)
download_dataset("https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/t10k-images-idx3-ubyte", test_path)


In [3]:
import mindspore.dataset as ds
import mindspore.dataset.transforms.c_transforms as C
import mindspore.dataset.vision.c_transforms as CV
from mindspore.dataset.vision import Inter
from mindspore import dtype as mstype


In [4]:
def create_dataset(data_path, batch_size=32, repeat_size=1,
                   num_parallel_workers=1):
    # Define the dataset.
    mnist_ds = ds.MnistDataset(data_path)
    resize_height, resize_width = 32, 32
    rescale = 1.0 / 255.0
    shift = 0.0
    rescale_nml = 1 / 0.3081
    shift_nml = -1 * 0.1307 / 0.3081

    # Define the mapping to be operated.
    resize_op = CV.Resize((resize_height, resize_width), interpolation=Inter.LINEAR)
    rescale_nml_op = CV.Rescale(rescale_nml, shift_nml)
    rescale_op = CV.Rescale(rescale, shift)
    hwc2chw_op = CV.HWC2CHW()
    type_cast_op = C.TypeCast(mstype.int32)

    # Use the map function to apply data operations to the dataset.
    mnist_ds = mnist_ds.map(operations=type_cast_op, input_columns="label", num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(operations=[resize_op, rescale_op, rescale_nml_op, hwc2chw_op], input_columns="image", num_parallel_workers=num_parallel_workers)


    # Perform shuffle, batch and repeat operations.
    buffer_size = 10000
    mnist_ds = mnist_ds.shuffle(buffer_size=buffer_size)
    mnist_ds = mnist_ds.batch(batch_size, drop_remainder=True)
    mnist_ds = mnist_ds.repeat(count=repeat_size)

    return mnist_ds


In [5]:
import mindspore.nn as nn
from mindspore.common.initializer import Normal
from mindspore.ops import ImageSummary, TensorSummary

class LeNet5(nn.Cell):
    """
    Lenet network structure
    """
    def __init__(self, num_class=10, num_channel=1):
        super(LeNet5, self).__init__()
        # Define the required operation.
        self.conv1 = nn.Conv2d(num_channel, 6, 5, pad_mode='valid')
        self.conv2 = nn.Conv2d(6, 16, 5, pad_mode='valid')
        self.fc1 = nn.Dense(16 * 5 * 5, 120, weight_init=Normal(0.02))
        self.fc2 = nn.Dense(120, 84, weight_init=Normal(0.02))
        self.fc3 = nn.Dense(84, num_class, weight_init=Normal(0.02))
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.image_summary = ImageSummary()
        self.tensor_summary = TensorSummary()

    def construct(self, x):
        # Use the defined operation to construct a forward network.
        self.image_summary("image", x)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        
        self.tensor_summary("tensor", x)
        return x

# Instantiate the network.
net = LeNet5()

In [6]:
# Define the loss function.
net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')

# Define the optimizer.
net_opt = nn.Momentum(net.trainable_params(), learning_rate=0.01, momentum=0.9)


In [7]:
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig
# Set model saving parameters.
config_ck = CheckpointConfig(save_checkpoint_steps=1875, keep_checkpoint_max=10)
# Use model saving parameters.
ckpoint = ModelCheckpoint(prefix="checkpoint_lenet", config=config_ck)


In [8]:
# Import the library required for model training.
from mindspore.nn import Accuracy
from mindspore.train.callback import LossMonitor
from mindspore import Model


In [9]:
def train_net(model, epoch_size, data_path, repeat_size, ckpoint_cb, sink_mode):
    """Define a training method."""
    # Load the training dataset.
    
    summary_collector = SummaryCollector(summary_dir = "./summary_dir", collect_freq = 1, collect_tensor_freq = 200)
    ds_train = create_dataset(os.path.join(data_path, "train"), 32, repeat_size)
    model.train(epoch_size, ds_train, callbacks=[ckpoint_cb, LossMonitor(125), summary_collector], dataset_sink_mode=sink_mode)

In [10]:
def test_net(model, data_path):
    """Define a validation method."""
    ds_eval = create_dataset(os.path.join(data_path, "test"))
    acc = model.eval(ds_eval, dataset_sink_mode=False)
    print("{}".format(acc))

In [11]:
from mindspore.profiler import Profiler
profiler = Profiler(output_path = './summary_dir/profiler_data')

train_epoch = 1
mnist_path = "./datasets/MNIST_Data"
dataset_size = 1
model = Model(net, net_loss, net_opt, metrics={"Accuracy": Accuracy()})
train_net(model, train_epoch, mnist_path, dataset_size, ckpoint, False)
profiler.analyse()
test_net(model, mnist_path)

[WARNING] ME(4707:140579900187264,MainProcess):2022-05-09-15:42:55.120.553 [mindspore/profiler/profiling.py:916] The target dir already exists. There may be some old profiling data, and they will be rewritten in the end.
[WARNING] DEBUG(4707,7fdb4f036280,python):2022-05-09-15:42:56.393.956 [mindspore/ccsrc/debug/debugger/debugger.cc:95] Debugger] Not enabling debugger. Debugger does not support CPU.


epoch: 1 step: 125, loss is 2.3118984699249268
epoch: 1 step: 250, loss is 2.2771129608154297
epoch: 1 step: 375, loss is 2.296725273132324
epoch: 1 step: 500, loss is 2.307905435562134
epoch: 1 step: 625, loss is 2.2862255573272705
epoch: 1 step: 750, loss is 2.2942287921905518
epoch: 1 step: 875, loss is 2.276190757751465
epoch: 1 step: 1000, loss is 0.33888059854507446
epoch: 1 step: 1125, loss is 0.11961334943771362
epoch: 1 step: 1250, loss is 0.36674317717552185
epoch: 1 step: 1375, loss is 0.11643192917108536
epoch: 1 step: 1500, loss is 0.0693666860461235
epoch: 1 step: 1625, loss is 0.08808504790067673
epoch: 1 step: 1750, loss is 0.05339649319648743
epoch: 1 step: 1875, loss is 0.1992499977350235


[ERROR] ME(4707:140579900187264,MainProcess):2022-05-09-15:43:40.440.615 [mindspore/train/callback/_summary_collector.py:592] Write meta data /home/andrey/project/Lab1/summary_dir/ckpt_dir/train_metadata.json failed, detail: [Errno 13] Permission denied: '/home/andrey/project/Lab1/summary_dir/ckpt_dir/train_metadata.json'
[WARNING] MD(4707,7fdb4f036280,python):2022-05-09-15:43:48.592.172 [mindspore/ccsrc/minddata/dataset/engine/consumers/tree_consumer.cc:68] RegisterProfilingManager] Dataset Profiling is already enabled for a different data pipeline.


{'Accuracy': 0.9622395833333334}


In [12]:
!mindinsight start --summary-base-dir ./summary_dir


Reusing MindInsight on port 8080. (pid 4401, Use "mindinsight stop --port 8080" to stop it.) Web address: http://127.0.0.1:8080


In [13]:
!mindinsight stop --port 8080

Stop mindinsight service successfully


In [14]:
from mindspore import load_checkpoint, load_param_into_net
# Load the saved model for testing.
param_dict = load_checkpoint("checkpoint_lenet-1_1875.ckpt")
# Load parameters to the network.
load_param_into_net(net, param_dict)

[WARNING] ME(4707:140579900187264,MainProcess):2022-05-09-15:43:54.457.613 [mindspore/common/parameter.py:337] The parameter definition is deprecated.
Please set a unique name for the parameter 'Parameter (name=Parameter, shape=(6, 1, 5, 5), dtype=Float32, requires_grad=True)'.
[WARNING] ME(4707:140579900187264,MainProcess):2022-05-09-15:43:54.459.496 [mindspore/common/parameter.py:337] The parameter definition is deprecated.
Please set a unique name for the parameter 'Parameter (name=Parameter, shape=(16, 6, 5, 5), dtype=Float32, requires_grad=True)'.
[WARNING] ME(4707:140579900187264,MainProcess):2022-05-09-15:43:54.462.173 [mindspore/common/parameter.py:337] The parameter definition is deprecated.
Please set a unique name for the parameter 'Parameter (name=Parameter, shape=(120, 400), dtype=Float32, requires_grad=True)'.
[WARNING] ME(4707:140579900187264,MainProcess):2022-05-09-15:43:54.464.816 [mindspore/common/parameter.py:337] The parameter definition is deprecated.
Please set a 

[]

In [15]:
import numpy as np
from mindspore import Tensor

# Define a test dataset. If batch_size is set to 1, an image is obtained.
ds_test = create_dataset(os.path.join(mnist_path, "test"), batch_size=1).create_dict_iterator()
data = next(ds_test)

# `images` indicates the test image, and `labels` indicates the actual classification of the test image.
images = data["image"].asnumpy()
labels = data["label"].asnumpy()

# Use the model.predict function to predict the classification of the image.
output = model.predict(Tensor(data['image']))
predicted = np.argmax(output.asnumpy(), axis=1)

# Output the predicted classification and the actual classification.
print(f'Predicted: "{predicted[0]}", Actual: "{labels[0]}"')


[WARNING] MD(4707,7fdb4f036280,python):2022-05-09-15:43:54.540.671 [mindspore/ccsrc/minddata/dataset/engine/consumers/tree_consumer.cc:68] RegisterProfilingManager] Dataset Profiling is already enabled for a different data pipeline.


Predicted: "2", Actual: "2"
